당신은 모델을 만들었습니다. 그런데 얼마나 좋은가요?

이번 과정에서는 모델의 성능을 평가하는 방법을 배웁니다. 모델 성능 측정은 모델을 반복적으로 개선하는 열쇠입니다.

# 모델 평가란?

당신은 제작하는 거의 모든 모델을 평가하고 싶을 것입니다. (전부는 아니지만) 대부분의 경우 모델의 성능을 평가하는 지표는 예측 정확도입니다. 다시 말해, 모델의 예측은 실제에 가까워질 것입니다.

많은 사람들이 예측 정확도를 측정할 때 큰 실수를 합니다. 그들은 *훈련 데이터*를 이용해 예측을 하고 이러한 예측을 *훈련 데이터*의 목표 값과 비교합니다. 이러한 접근 방식의 문제점과 해결법은 잠시 후에 알아보겠지만, 먼저 이 방법을 어떻게 해야 할지 생각해 보겠습니다.

우선 모델 성능을 이해하기 쉬운 방법으로 요약해야 합니다. 주택 10,000채에 대한 예측값과 실제 주택값을 비교해보면 좋고 나쁜 예측값이 뒤섞여 있을 것입니다. 10,000개의 예측값과 실제값 목록을 살펴보는 것은 무의미할 것입니다. 우리는 이를 하나의 지표로 나타내야 합니다.

모델 성능을 요약하는 지표는 많지만 **평균 절대 오차(Mean Absolute Error)** 로 (혹은 **MAE**) 시작하겠습니다. 이 지표의 마지막 단어인 오차부터 살펴보겠습니다.

각 주택의 예측 오차는 다음과 같습니다: <br>
```
error=actual-predicted
```

따라서 만약 주택 가격이 \$150,000이고 \$100,000으로 예측했다면 오차는 \$50,000입니다.

MAE 지표를 사용하면 각 오차의 절댓값을 가져옵니다. 이는 각 오차를 양수값으로 변환합니다. 그 후 오차들의 평균을 구합니다. 이것이 우리의 모델 성능을 평가하는 지표입니다. 이를 쉽게 말하면 다음과 같습니다.

> 평균적으로, 우리의 예측은 약 X만큼 빗나갔습니다.

MAE를 계산하기 위해서는 우선 모델이 필요합니다.

In [1]:
import pandas as pd

# 데이터를 불러옵니다
melbourne_file_path = '../../data/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# 결측값이 포함된 행을 제거합니다
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# 목표값과 특성을 선택합니다
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor
# 모델을 정의합니다
melbourne_model = DecisionTreeRegressor()
# 모델 피팅
melbourne_model.fit(X, y)

DecisionTreeRegressor()

모델을 만든 후, 평균 절대 오차를 계산하는 방법은 다음과 같습니다:

In [2]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

# "표본 내" 점수의 문제점

방금 계산한 척도는 "표본 내" 점수라고 할 수 있습니다. 우리는 모델을 만들고 평가하기 위해 단일 "표본"을 사용했습니다. 이것이 나쁜 이유입니다.

큰 부동산 시장에서 문의 색은 집값과 무관합니다.

그러나 모델을 만들 때 사용한 표본에서 녹색 문이 있는 주택은 모두 비쌌습니다. 모델의 일은 집값을 예측하는 규칙을 찾는 것이므로 이러한 규칙을 찾을 것이고, 따라서 녹색 문이 있는 집은 언제나 높은 가격으로 예측할 것입니다.

이러한 규칙은 훈련 데이터에서 파생된 것이므로 훈련 데이터에 대해 모델은 높은 정확도를 보일 것입니다.

그러나 모델이 새로운 데이터를 접할 때 이러한 규칙이 유지되지 않는다면 실제로 사용할 때는 매우 부정확할 것입니다.

모델의 실제 가치는 새로운 데이터를 예측하는 것에서 비롯되므로 모델을 구축하는 데 사용되지 않은 데이터에 대한 성능을 측정합니다. 가장 간단한 방법은 모델을 제작하기 전에 일부 데이터를 제외시킨 다음, 모델의 정확도 측정에 사용하는 것입니다. 이러한 데이터를 **검증 데이터(validation data)** 라고 합니다.


# 코딩


Scikit-learn 라이브러리는 데이터를 두 묶음으로 나누는 `train_test_split`라는 함수를 지원합니다. 우리는 일부 데이터를 이용해 모델을 피팅하고, 나머지 데이터를 검증 데이터로써 사용해 `mean_absolute_error`를 계산할 것입니다.

다음은 이를 수행하는 코드입니다:

In [3]:
from sklearn.model_selection import train_test_split

# 특성과 목표 데이터 모두를 훈련 및 검증 데이터로 나눕니다
# 분리는 난수 생성기에 기반합니다
# random_state를 특정지으면 언제나 같은 묶음으로 나뉨을 보장할 수 있습니다
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# 모델 정의
melbourne_model = DecisionTreeRegressor()
# 모델 피팅
melbourne_model.fit(train_X, train_y)

# 검증 데이터에 대한 예측
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

263185.5422853454


# 와!

표본 내 데이터에 대한 평균 절대 오차는 약 500 달러였습니다. 표본 외 데이터의 오차는 250,000 달러 이상입니다.

이것이 거의 정확히 맞는 모델과 실용적인 용도로 사용할 수 없는 모델의 차이입니다. 참고로 검증 데이터의 평균 집값은 110만 달러입니다. 새로운 데이터의 오차는 평균 집값의 약 4분의 1인 것입니다.

더 나은 특성을 찾거나 다른 모델 유형을 사용하는 등 모델 성능을 개선하는 방법은 여러 가지입니다.